In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import seaborn as sns
df = pd.read_excel('/content/drive/MyDrive/Cookstove_Plan15_for_sharing/data/charc_firewood_baseline_adjusted_merged.xlsx')
VERs_all=pd.read_excel('/content/drive/MyDrive/Cookstove_Plan15_for_sharing/results/VERS_merged.xlsx')

In [3]:
## SY added
df['protocol_type'] = df['protocol_type'].replace('CDM AMS-I-E','CDM AMS I E')
df['protocol_type'] = df['protocol_type'].replace('CDM AMS II.G','CDM AMS II G')

In [4]:
df.head(3)

,protocol_type,protocol_version,PoA,protocol_id,monitoring_period,stove_year_cohort,total_verified_credits,type,verified_credits,verified_credits_per_stove,...,project_adoption_commercial,project_usage_commercial,leakage_binary,leakage_default,leakage_tCO2e,more_than_one_stove_adj,rebound,other_tCO2e_reduction,project_stacking,project_stacking_commercial
0,Units,0,0,0,0,0,tCO2e,0,tCO2e,tCO2e/stove/day,...,%,%,0,%,tCO2e,%,0.00,0,0.0,0
1,CDM AMS II G,AMS-II.G. ver. 9,10471,PoA 10471 CPA1,MR1,Domestic (2020),65200,firewood,65200,0.003652,...,0,0,1,0.95,0,1,0.78,0,0.0,0
2,CDM AMS II G,AMS-II.G. ver. 9,10471,PoA 10471 CPA2,MR1,Domestic (2020),63603,firewood,63603,0.003642,...,0,0,1,0.95,0,1,0.78,0,0.0,0


In [5]:
unique_stoves = df['stove'].unique()
for stove in unique_stoves[1:]:
    var_name = f"{stove.lower()}_VERs"
    globals()[var_name] = df.loc[df['stove'] == stove, 'verified_credits_per_stove'].sum()
    globals()[f"{stove}_N"] = 0
    globals()[f"{stove}_VERs_agw"] = []
    globals()[f"{stove}_VERs"] = list() ## updated
    globals()[f"{stove}_ratio_analysis_project_mean_compilation"] = []
    globals()[f"{stove}_ratio_project_analysis_mean_compilation"] = []
    globals()[f"{stove}_ratio_analysis_project_lower_compilation"] = []
    globals()[f"{stove}_ratio_analysis_project_upper_compilation"] = []
    globals()[f"{stove}_ratio_project_analysis_lower_compilation"] = []
    globals()[f"{stove}_ratio_project_analysis_upper_compilation"] = []

# create variables for each country
unique_countries = df['country'].unique()
for country in unique_countries[1:]:
    var_name = f"{country.lower()}_VERs"
    globals()[var_name] = df.loc[df['country'] == country, 'verified_credits_per_stove'].sum()
    # lower the country name
    var_base_name = country.lower().replace(" ", "").replace("-", "").replace("'", "").replace(" ", "")  # lower the country name
    globals()[f"{var_base_name}_VERs_agw"] = []  # 'country_VERs_agw' list
    globals()[f"{var_base_name}_VERs"] = list()  # 'country_VERs' list (added)
    globals()[f"{var_base_name}_N"] = 0 # 'country_N'
    globals()[f"{var_base_name}_ratio_analysis_project_mean_compilation"] = [] # 'country_ratio_analysis_project_mean_compilation' list
    globals()[f"{var_base_name}_ratio_project_analysis_mean_compilation"] = [] # 'country_ratio_project_analysis_mean_compilation' list
    globals()[f"{var_base_name}_ratio_analysis_project_lower_compilation"] = [] # 'country_ratio_analysis_project_lower_compilation' list
    globals()[f"{var_base_name}_ratio_analysis_project_upper_compilation"] = []   # 'country_ratio_analysis_project_upper_compilation' list
    globals()[f"{var_base_name}_ratio_project_analysis_lower_compilation"] = [] # 'country_ratio_project_analysis_lower_compilation' list
    globals()[f"{var_base_name}_ratio_project_analysis_upper_compilation"] = [] #  'country_ratio_project_analysis_upper_compilation' list

# create variables for each type
unique_types = df['type'].unique()

for type1 in unique_types[1:]:
    globals()[f"{type1}_N"] = 0
    globals()[f"{type1}_VERs"] = list()
    globals()[f"{type1}_VERs_agw"] = []
    globals()[f"{type1}_ratio_analysis_project_mean_compilation"] = []
    globals()[f"{type1}_ratio_project_analysis_mean_compilation"] = []
    globals()[f"{type1}_ratio_analysis_project_lower_compilation"] = []
    globals()[f"{type1}_ratio_analysis_project_upper_compilation"] = []
    globals()[f"{type1}_ratio_project_analysis_lower_compilation"] = []
    globals()[f"{type1}_ratio_project_analysis_upper_compilation"] = []

# create lists for each protocol_type
unique_protocol_types = df['protocol_type'].unique()

for protocol_type in unique_protocol_types[1:]:
  var_protocol_type = protocol_type.replace(" ", "_").replace("_AMS", "")
  globals()[f"{var_protocol_type}_VERs_agw"] = list()
  globals()[f"{var_protocol_type}_ratio_analysis_project_mean_compilation"] = []
  globals()[f"{var_protocol_type}_ratio_project_analysis_mean_compilation"] = []
  globals()[f"{var_protocol_type}_ratio_analysis_project_lower_compilation"] = []
  globals()[f"{var_protocol_type}_ratio_analysis_project_upper_compilation"] = []
  globals()[f"{var_protocol_type}_ratio_project_analysis_lower_compilation"] = []
  globals()[f"{var_protocol_type}_ratio_project_analysis_upper_compilation"] = []

In [6]:
ALL_N = 0
all_VERS_agw = []
ratio_analysis_project_mean_compilation=[]
ratio_project_analysis_mean_compilation=[]
ratio_analysis_project_lower_compilation=[]
ratio_analysis_project_upper_compilation=[]
ratio_project_analysis_lower_compilation=[]
ratio_project_analysis_upper_compilation=[]

# For protocol wide over-estimation
GS_Meter_N=0
GS_N=0
CDM_II_G_N=0
CDM_I_E_N=0

In [7]:
#compiling our results on a protocol basis
all_VERS_agw=[]
GS_Meter_VERs_agw=[]
GS_VERs_agw=[]
CDM_II_G_VERs_agw=[]
CDM_I_E_VERs_agw=[]

In [8]:
#import relevant packages
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

from scipy import stats as stats
from scipy.stats import triang

import seaborn as sns

#triangular distributions

np.random.seed(4)
N = 10000                # random numbers to generate


#Adoption
min_adoption=.4
mean_adoption=.58
max_adoption= .92
n_adoption=np.random.triangular(left=min_adoption, mode=mean_adoption, right=max_adoption, size=(N))

req_adoption=np.random.uniform(0,mean_adoption,N)
#Usage
min_usage=0.16
mean_usage=0.52
max_usage=0.85
n_usage=np.random.triangular(left=min_usage, mode=mean_usage, right=max_usage, size=(N))
req_usage=np.random.uniform(0,mean_usage,N)

#Stacking
min_stacking=.193
mean_stacking=.673
max_stacking=.99
n_stacking=np.random.triangular(left=min_stacking, mode=mean_stacking, right=max_stacking, size=(N))
req_stacking=np.random.uniform(0,mean_stacking,N)

# Renewable GWP from Whitman & Lehman

gwp_ch4_renewable=28 #tCO2e/tonne CH4 i.e., instead of typically 25

# Whitman & lehmann 2011 also cite a value for CO
# but we do not include CO in our analysis

gwp_ch4_agw=30
gwp_n2O_agw=298

eff_trad_firewood=0.14
eff_improv_firewood=.21
eff_trad_charcoal=0.2
eff_improv_charcoal=.22
eff_kerosene=.47
eff_LPG=.5
eff_ethanol=.47
eff_pellets=.5 #only two pellet projects is the mimi motto

ncv_firewood_agw=.016
ncv_charcoal_agw=.027
ncv_kerosene_agw=.043
ncv_LPG_agw=.045
ncv_ethanol_agw=.027
ncv_pellets_agw=.017

kg_per_liter_ethanol_agw=0.783 #kg/liter

ef_combustion_co2_firewood=95.1 #tCO2/TJ
ef_combustion_ch4_firewood=0.58 #tCH4/TJ
ef_combustion_n2O_firewood=0 #tN2O/TJ

ef_combustion_co2_charcoal=114.75 #tCO2/TJ
ef_combustion_ch4_charcoal=0.255 #tCH4/TJ
ef_combustion_n2O_charcoal=0 #tN2O/TJ


ef_total_not_combustion_co2_charcoal=87.725 #tCO2/TJ
ef_total_not_combustion_ch4_charcoal=2.06 #tCH4/TJ
ef_total_not_combustion_n2O_charcoal=0.0075#tN2O/TJ

ef_total_co2_kerosene=79.9 #tCO2/TJ
ef_total_ch4_kerosene=.107 #tCH4/TJ
ef_total_n2O_kerosene=0.000141 #tN2O/TJ

ef_total_co2_LPG=82.8#tCO2/TJ
ef_total_ch4_LPG=0.117 #tCH4/TJ
ef_total_n2O_LPG=0.000188 #tN2O/TJ

ef_total_co2_ethanol=60.45#tCO2/TJ
ef_total_ch4_ethanol=0.61 #tCH4/TJ
ef_total_n2O_ethanol=0.05 #tN2O/TJ

ef_total_co2_pellets=90.05#tCO2/TJ
ef_total_ch4_pellets=0.105#tCH4/TJ
ef_total_n2O_pellets=0 #tN2O/TJ

ef_total_zeroed_co2_ethanol=-9.78#tCO2/TJ
ef_total_zeroed_ch4_ethanol=0.61 #tCH4/TJ
ef_total_zeroed_n2O_ethanol=0.05 #tN2O/TJ

ef_total_zeroed_co2_pellets=0 #tCO2/TJ
ef_total_zeroed_ch4_pellets=0.105 #tCH4/TJ
ef_total_zeroed_n2O_pellets=0 #tN2O/TJ

sum_VERS_not_per_stove_per_day_agw=0

In [9]:
i=1 # The row you want to evaluate
while i<len(df):
    # first row is units
    VERs=VERs_all.iloc[i,1]
    protocol_type=df['protocol_type'][i]

    protocol_id=df['protocol_id'][i]

    monitoring_period=df['monitoring_period'][i]

    verified_credits=df['verified_credits'][i]

    num_countries=df['num_countries'][i]

    prov_project=df['prov_project'][i]

    fNRB_project=df['fNRB_project'][i]

    mofus_fNRB_bailis=df['mofus_fNRB_bailis'][i]


    n_devices_domestic=df['n_devices_domestic'][i]

    n_devices_commercial=df['n_devices_commercial'][i]

    project_days=df['project_days'][i]

    project_days_commercial=df['project_days_commercial'][i]

    av_hhsize=df['av_hhsize'][i]


    stove_year_cohort=df['stove_year_cohort'][i]

    # Baseline Fuel % What are the baseline fuels in this project location? [put 0% if no use in this location]
    prc_firewood_baseline=df['prc_firewood_baseline'][i]

    prc_charcoal_baseline=df['prc_charcoal_baseline'][i]

    prc_kerosene_baseline=df['prc_kerosene_baseline'][i]

    prc_lpg_baseline=df['prc_lpg_baseline'][i]

    prc_ethanol_baseline=df['prc_ethanol_baseline'][i]

    prc_electric_baseline=df['prc_electric_baseline'][i]

    prc_pellets_baseline=df['prc_pellets_baseline'][i]





    # Baseline Fuel % What are the baseline commercial fuels in this project location? [put 0% if no use in this location]
    prc_firewood_baseline_commercial=df['prc_firewood_baseline_commercial'][i]

    prc_charcoal_baseline_commercial=df['prc_charcoal_baseline_commercial'][i]

    prc_kerosene_baseline_commercial=df['prc_kerosene_baseline_commercial'][i]

    prc_lpg_baseline_commercial=df['prc_lpg_baseline_commercial'][i]

    prc_ethanol_baseline_commercial=df['prc_ethanol_baseline_commercial'][i]

    prc_electric_baseline_commercial=df['prc_electric_baseline_commercial'][i]

    prc_pellets_baseline_commercial=df['prc_pellets_baseline_commercial'][i]






    # Baseline Consumption: What is the baseline fuel consumption of each baseline fuel in this project location [put 0 if no use in this location]
    kg_firewood_baseline=df['kg_firewood_baseline'][i]

    kg_charcoal_baseline=df['kg_charcoal_baseline'][i]

    kg_kerosene_baseline=df['kg_kerosene_baseline'][i]

    kg_lpg_baseline=df['kg_lpg_baseline'][i]

    l_ethanol_baseline=df['l_ethanol_baseline'][i]

    kg_per_liter_ethanol_baseline=df['kg_per_liter_ethanol_baseline'][i]

    kWh_electric_baseline=df['kWh_electric_baseline'][i]

    kg_pellets_baseline=df['kg_pellets_baseline'][i]


    kg_firewood_baseline_commercial=df['kg_firewood_baseline_commercial'][i]

    kg_charcoal_baseline_commercial=df['kg_charcoal_baseline_commercial'][i]

    kg_kerosene_baseline_commercial=df['kg_kerosene_baseline_commercial'][i]

    kg_lpg_baseline_commercial=df['kg_lpg_baseline_commercial'][i]

    l_ethanol_baseline_commercial=df['l_ethanol_baseline_commercial'][i]

    kWh_electric_baseline_commercial=df['kWh_electric_baseline_commercial'][i]

    kg_pellets_baseline_commercial=df['kg_pellets_baseline_commercial'][i]


    # Stove Efficiencies at Baseline


    eff_firewood_baseline=df['eff_firewood_baseline'][i]

    eff_charcoal_baseline=df['eff_charcoal_baseline'][i]

    eff_kerosene_baseline=df['eff_kerosene_baseline'][i]

    eff_LPG_baseline=df['eff_LPG_baseline'][i]

    eff_ethanol_baseline=df['eff_ethanol_baseline'][i]

    eff_electric_baseline=df['eff_electric_baseline'][i]

    eff_pellets_baseline=df['eff_pellets_baseline'][i]


    # Commercial Baseline Stove Efficiences

    eff_firewood_baseline_commercial=df['eff_firewood_baseline_commercial'][i]

    eff_charcoal_baseline_commercial=df['eff_charcoal_baseline_commercial'][i]

    eff_kerosene_baseline_commercial=df['eff_kerosene_baseline_commercial'][i]

    eff_LPG_baseline_commercial=df['eff_LPG_baseline_commercial'][i]

    eff_ethanol_baseline_commercial=df['eff_ethanol_baseline_commercial'][i]

    eff_electric_baseline_commercial=df['eff_electric_baseline_commercial'][i]

    eff_pellet_baseline_commercial=df['eff_pellet_baseline_commercial'][i]


    # Caloric Values: What are the assumed calorific values  of the baseline fuels? (TJ/ton) [put 0 if not in use in this location]

    ncv_firewood_baseline=df['ncv_firewood_baseline'][i]

    ncv_charcoal_baseline=df['ncv_charcoal_baseline'][i]

    ncv_kerosene_baseline=df['ncv_kerosene_baseline'][i]

    ncv_LPG_baseline=df['ncv_LPG_baseline'][i]

    ncv_ethanol_baseline=df['ncv_ethanol_baseline'][i]

    ncv_pellets_baseline=df['ncv_pellets_baseline'][i]


    # CO2 Emission Factors: What is the assumed carbon dioxide emission factor of the baseline fuels?
    #[if CDM write in the assumed EF of the projected fossil fuel]
    #[ put 0 if you do not account for CO2 EF at Baseline or this fuel is not applicable for the project]
    co2_ef_firewood=df['co2_ef_firewood'][i]
    co2_ef_charcoal=df['co2_ef_charcoal'][i]

    co2_ef_kerosene=df['co2_ef_kerosene'][i]

    co2_ef_LPG=df['co2_ef_LPG'][i]

    co2_ef_ethanol=df['co2_ef_ethanol'][i]

    co2_ef_electric=df['co2_ef_electric'][i]

    co2_ef_pellets=df['co2_ef_pellets'][i]


    #Non-CO2 Baseline Emission Factors

    nonco2_ef_firewood=df['nonco2_ef_firewood'][i]
    nonco2_ef_charcoal=df['nonco2_ef_charcoal'][i]

    nonco2_ef_kerosene=df['nonco2_ef_kerosene'][i]

    nonco2_ef_LPG=df['nonco2_ef_LPG'][i]

    nonco2_ef_ethanol=df['nonco2_ef_ethanol'][i]

    nonco2_ef_electric=df['nonco2_ef_electric'][i]

    nonco2_ef_pellets=df['nonco2_ef_pellets'][i]


    # Emissions from Stove Production

    stoveproduction_ef_co2_firewood_baseline= df['stoveproduction_ef_co2_firewood_baseline'][i]
    stoveproduction_ef_ch4_firewood_baseline= df['stoveproduction_ef_ch4_firewood_baseline'][i]
    stoveproduction_ef_n2O_firewood_baseline= df['stoveproduction_ef_n2O_firewood_baseline'][i]


    stoveproduction_ef_co2_charcoal_baseline=df['stoveproduction_ef_co2_charcoal_baseline'][i]
    stoveproduction_ef_ch4_charcoal_baseline=df['stoveproduction_ef_ch4_charcoal_baseline'][i]
    stoveproduction_ef_n2O_charcoal_baseline=df['stoveproduction_ef_n2O_charcoal_baseline'][i]


    stoveproduction_ef_co2_kerosene_baseline=df['stoveproduction_ef_co2_kerosene_baseline'][i]
    stoveproduction_ef_ch4_kerosene_baseline=df['stoveproduction_ef_ch4_kerosene_baseline'][i]
    stoveproduction_ef_n2O_kerosene_baseline=df['stoveproduction_ef_n2O_kerosene_baseline'][i]


    stoveproduction_ef_co2_LPG_baseline=df['stoveproduction_ef_co2_LPG_baseline'][i]
    stoveproduction_ef_ch4_LPG_baseline=df['stoveproduction_ef_ch4_LPG_baseline'][i]
    stoveproduction_ef_n2O_LPG_baseline=df['stoveproduction_ef_n2O_LPG_baseline'][i]


    stoveproduction_ef_co2_ethanol_baseline=df['stoveproduction_ef_co2_ethanol_baseline'][i]
    stoveproduction_ef_ch4_ethanol_baseline=df['stoveproduction_ef_ch4_ethanol_baseline'][i]
    stoveproduction_ef_n2O_ethanol_baseline=df['stoveproduction_ef_n2O_ethanol_baseline'][i]


    stoveproduction_ef_co2_electric_baseline=df['stoveproduction_ef_co2_electric_baseline'][i]
    stoveproduction_ef_ch4_electric_baseline=df['stoveproduction_ef_ch4_electric_baseline'][i]
    stoveproduction_ef_n2O_electric_baseline=df['stoveproduction_ef_n2O_electric_baseline'][i]


    stoveproduction_ef_co2_pellets_baseline=df['stoveproduction_ef_co2_pellets_baseline'][i]
    stoveproduction_ef_ch4_pellets_baseline=df['stoveproduction_ef_ch4_pellets_baseline'][i]
    stoveproduction_ef_n2O_pellets_baseline=df['stoveproduction_ef_n2O_pellets_baseline'][i]


    feedstockproduction_ef_co2_firewood_baseline=df['feedstockproduction_ef_co2_firewood_baseline'][i]
    feedstockproduction_ef_ch4_firewood_baseline=df['feedstockproduction_ef_ch4_firewood_baseline'][i]
    feedstockproduction_ef_n2O_firewood_baseline=df['feedstockproduction_ef_n2O_firewood_baseline'][i]


    feedstockproduction_ef_co2_charcoal_baseline=df['feedstockproduction_ef_co2_charcoal_baseline'][i]
    feedstockproduction_ef_ch4_charcoal_baseline=df['stoveproduction_ef_ch4_charcoal_baseline'][i]
    feedstockproduction_ef_n2O_charcoal_baseline=df['feedstockproduction_ef_n2O_charcoal_baseline'][i]


    feedstockproduction_ef_co2_kerosene_baseline=df['feedstockproduction_ef_co2_kerosene_baseline'][i]
    feedstockproduction_ef_ch4_kerosene_baseline=df['feedstockproduction_ef_ch4_kerosene_baseline'][i]
    feedstockproduction_ef_n2O_kerosene_baseline=df['feedstockproduction_ef_n2O_kerosene_baseline'][i]


    feedstockproduction_ef_co2_LPG_baseline=df['feedstockproduction_ef_co2_LPG_baseline'][i]
    feedstockproduction_ef_ch4_LPG_baseline=df['feedstockproduction_ef_ch4_LPG_baseline'][i]
    feedstockproduction_ef_n2O_LPG_baseline=df['feedstockproduction_ef_n2O_LPG_baseline'][i]


    feedstockproduction_ef_co2_ethanol_baseline=df['feedstockproduction_ef_co2_firewood_baseline'][i]
    feedstockproduction_ef_ch4_ethanol_baseline=df['feedstockproduction_ef_ch4_firewood_baseline'][i]
    feedstockproduction_ef_n2O_ethanol_baseline=df['feedstockproduction_ef_n2O_firewood_baseline'][i]


    feedstockproduction_ef_co2_electric_baseline=df['feedstockproduction_ef_co2_firewood_baseline'][i]
    feedstockproduction_ef_ch4_electric_baseline=df['feedstockproduction_ef_ch4_firewood_baseline'][i]
    feedstockproduction_ef_n2O_electric_baseline=df['feedstockproduction_ef_n2O_firewood_baseline'][i]


    feedstockproduction_ef_co2_pellets_baseline=df['feedstockproduction_ef_co2_firewood_baseline'][i]
    feedstockproduction_ef_ch4_pellets_baseline=df['feedstockproduction_ef_ch4_firewood_baseline'][i]
    feedstockproduction_ef_n2O_pellets_baseline=df['feedstockproduction_ef_n2O_firewood_baseline'][i]


    distribution_ef_co2_firewood_baseline=df['distribution_ef_co2_firewood_baseline'][i]
    distribution_ef_ch4_firewood_baseline=df['distribution_ef_ch4_firewood_baseline'][i]
    distribution_ef_n2O_firewood_baseline=df['distribution_ef_n2O_firewood_baseline'][i]


    distribution_ef_co2_charcoal_baseline=df['distribution_ef_co2_charcoal_baseline'][i]
    distribution_ef_ch4_charcoal_baseline=df['distribution_ef_ch4_charcoal_baseline'][i]
    distribution_ef_n2O_charcoal_baseline=df['distribution_ef_n2O_charcoal_baseline'][i]


    distribution_ef_co2_kerosene_baseline=df['distribution_ef_co2_kerosene_baseline'][i]
    distribution_ef_ch4_kerosene_baseline=df['distribution_ef_ch4_kerosene_baseline'][i]
    distribution_ef_n2O_kerosene_baseline=df['distribution_ef_n2O_kerosene_baseline'][i]


    distribution_ef_co2_LPG_baseline=df['distribution_ef_co2_LPG_baseline'][i]
    distribution_ef_ch4_LPG_baseline=df['distribution_ef_ch4_LPG_baseline'][i]
    distribution_ef_n2O_LPG_baseline=df['distribution_ef_n2O_LPG_baseline'][i]


    distribution_ef_co2_ethanol_baseline=df['distribution_ef_co2_ethanol_baseline'][i]
    distribution_ef_ch4_ethanol_baseline=df['distribution_ef_ch4_ethanol_baseline'][i]
    distribution_ef_n2O_ethanol_baseline=df['distribution_ef_n2O_ethanol_baseline'][i]


    distribution_ef_co2_electric_baseline=df['distribution_ef_co2_electric_baseline'][i]
    distribution_ef_ch4_electric_baseline=df['distribution_ef_ch4_electric_baseline'][i]
    distribution_ef_n2O_electric_baseline=df['distribution_ef_n2O_electric_baseline'][i]


    distribution_ef_co2_pellets_baseline=df['distribution_ef_co2_pellets_baseline'][i]
    distribution_ef_ch4_pellets_baseline=df['distribution_ef_ch4_pellets_baseline'][i]
    distribution_ef_n2O_pellets_baseline=df['distribution_ef_n2O_pellets_baseline'][i]


    processing_ef_co2_firewood_baseline=df['processing_ef_co2_firewood_baseline'][i]
    processing_ef_ch4_firewood_baseline=df['processing_ef_ch4_firewood_baseline'][i]
    processing_ef_n2O_firewood_baseline=df['processing_ef_n2O_firewood_baseline'][i]


    processing_ef_co2_charcoal_baseline=df['processing_ef_co2_firewood_baseline'][i]
    processing_ef_ch4_charcoal_baseline=df['processing_ef_co2_firewood_baseline'][i]
    processing_ef_n2O_charcoal_baseline=df['processing_ef_co2_firewood_baseline'][i]


    processing_ef_co2_kerosene_baseline=df['processing_ef_co2_kerosene_baseline'][i]
    processing_ef_ch4_kerosene_baseline=df['processing_ef_ch4_kerosene_baseline'][i]
    processing_ef_n2O_kerosene_baseline=df['processing_ef_n2O_kerosene_baseline'][i]


    processing_ef_co2_LPG_baseline=df['processing_ef_co2_LPG_baseline'][i]
    processing_ef_ch4_LPG_baseline=df['processing_ef_ch4_LPG_baseline'][i]
    processing_ef_n2O_LPG_baseline=df['processing_ef_n2O_LPG_baseline'][i]


    processing_ef_co2_ethanol_baseline=df['processing_ef_co2_ethanol_baseline'][i]
    processing_ef_ch4_ethanol_baseline=df['processing_ef_ch4_ethanol_baseline'][i]
    processing_ef_n2O_ethanol_baseline=df['processing_ef_n2O_ethanol_baseline'][i]


    processing_ef_co2_electric_baseline=df['processing_ef_co2_electric_baseline'][i]
    processing_ef_ch4_electric_baseline=df['processing_ef_ch4_electric_baseline'][i]
    processing_ef_n2O_electric_baseline=df['processing_ef_n2O_electric_baseline'][i]


    processing_ef_co2_pellets_baseline=df['processing_ef_co2_firewood_baseline'][i]
    processing_ef_ch4_pellets_baseline=df['processing_ef_ch4_firewood_baseline'][i]
    processing_ef_n2O_pellets_baseline=df['processing_ef_n2O_firewood_baseline'][i]

    # Project Fuel % What are the project domestic fuels in this project location? [put 0% if no use in this location]
    prc_firewood_project=df['prc_firewood_project'][i]

    prc_charcoal_project=df['prc_charcoal_project'][i]

    prc_kerosene_project=df['prc_kerosene_project'][i]

    prc_lpg_project=df['prc_lpg_project'][i]

    prc_ethanol_project=df['prc_ethanol_project'][i]

    prc_electric_project=df['prc_electric_project'][i]

    prc_pellets_project=df['prc_pellets_project'][i]



    # Project Fuel % What are the project commercial fuels in this project location? [put 0% if no use in this location]
    prc_firewood_project_commercial=df['prc_firewood_project_commercial'][i]

    prc_charcoal_project_commercial=df['prc_charcoal_project_commercial'][i]

    prc_kerosene_project_commercial=df['prc_kerosene_project_commercial'][i]

    prc_lpg_project_commercial=df['prc_lpg_project_commercial'][i]

    prc_ethanol_project_commercial=df['prc_ethanol_project_commercial'][i]

    prc_electric_project_commercial=df['prc_electric_project_commercial'][i]

    prc_pellets_project_commercial=df['prc_pellets_project_commercial'][i]



    # Project Fuel Consumption

    kg_firewood_project=df['kg_firewood_project'][i]

    kg_charcoal_project=df['kg_charcoal_project'][i]

    kg_kerosene_project=df['kg_kerosene_project'][i]

    kg_lpg_project=df['kg_lpg_project'][i]

    l_ethanol_project=df['l_ethanol_project'][i]


    kg_per_liter_ethanol_project=df['kg_per_liter_ethanol_project'][i]

    kWh_electric_project=df['kWh_electric_project'][i]

    kg_pellets_project=df['kg_pellets_project'][i]


    # Project Insitutional Consumption


    kg_firewood_project_commercial=df['kg_firewood_project_commercial'][i]

    kg_charcoal_project_commercial=df['kg_charcoal_project_commercial'][i]

    kg_kerosene_project_commercial=df['kg_kerosene_project_commercial'][i]

    kg_lpg_project_commercial=df['kg_lpg_project_commercial'][i]

    l_ethanol_project_commercial=df['l_ethanol_project_commercial'][i]

    kWh_electric_project_commercial=df['kWh_electric_project_commercial'][i]

    kg_pellets_project_commercial=df['kg_pellets_project_commercial'][i]




    # Project Stove Efficiencies

    eff_firewood_project=df['eff_firewood_project'][i]

    eff_charcoal_project=df['eff_charcoal_project'][i]

    eff_kerosene_project=df['eff_kerosene_project'][i]

    eff_LPG_project=df['eff_LPG_project'][i]

    eff_ethanol_project=df['eff_ethanol_project'][i]

    eff_electric_project=df['eff_electric_project'][i]

    eff_pellet_project=df['eff_pellet_project'][i]


    # Project Instituitonal Stove Efficiencies

    eff_firewood_project_commercial=df['eff_firewood_project_commercial'][i]

    eff_charcoal_project_commercial=df['eff_charcoal_project_commercial'][i]

    eff_kerosene_project_commercial=df['eff_kerosene_project_commercial'][i]

    eff_LPG_project_commercial=df['eff_LPG_project_commercial'][i]

    eff_ethanol_project_commercial=df['eff_ethanol_project_commercial'][i]

    eff_electric_project_commercial=df['eff_electric_project_commercial'][i]

    eff_pellet_project_commercial=df['eff_pellet_project_commercial'][i]


    #Assumed Caloric Values in the Project Scenario

    ncv_firewood_project=df['ncv_firewood_project'][i]

    ncv_charcoal_project=df['ncv_charcoal_project'][i]

    ncv_kerosene_project=df['ncv_kerosene_project'][i]

    ncv_LPG_project=df['ncv_LPG_project'][i]

    ncv_ethanol_project=df['ncv_ethanol_project'][i]

    ncv_pellets_project=df['ncv_pellets_project'][i]








    # CO2 Emission Factors in the Project

    co2_ef_firewood_project=df['co2_ef_firewood_project'][i]

    co2_ef_charcoal_project=df['co2_ef_charcoal_project'][i]

    co2_ef_kerosene_project=df['co2_ef_kerosene_project'][i]

    co2_ef_LPG_project=df['co2_ef_LPG_project'][i]

    co2_ef_ethanol_project=df['co2_ef_ethanol_project'][i]

    co2_ef_electric_project=df['co2_ef_electric_project'][i]

    co2_ef_pellets_project=df['co2_ef_pellets_project'][i]


    # Non-CO2 Emission Factors in the Project

    nonco2_ef_firewood_project=df['nonco2_ef_firewood_project'][i]

    nonco2_ef_charcoal_project=df['nonco2_ef_charcoal_project'][i]

    nonco2_ef_kerosene_project=df['nonco2_ef_kerosene_project'][i]

    nonco2_ef_LPG_project=df['nonco2_ef_LPG_project'][i]

    nonco2_ef_ethanol_project=df['nonco2_ef_ethanol_project'][i]

    nonco2_ef_electric_project=df['nonco2_ef_electric_project'][i]

    nonco2_ef_pellets_project=df['nonco2_ef_pellets_project'][i]

    # Stove Production Emission Factors


    stoveproduction_ef_co2_firewood_project=df['stoveproduction_ef_co2_firewood_project'][i]
    stoveproduction_ef_ch4_firewood_project=df['stoveproduction_ef_ch4_firewood_project'][i]
    stoveproduction_ef_n2O_firewood_project=df['stoveproduction_ef_n2O_firewood_project'][i]


    stoveproduction_ef_co2_charcoal_project=df['stoveproduction_ef_co2_charcoal_project'][i]
    stoveproduction_ef_ch4_charcoal_project=df['stoveproduction_ef_ch4_charcoal_project'][i]
    stoveproduction_ef_n2O_charcoal_project=df['stoveproduction_ef_n2O_charcoal_project'][i]


    stoveproduction_ef_co2_kerosene_project=df['stoveproduction_ef_co2_kerosene_project'][i]
    stoveproduction_ef_ch4_kerosene_project=df['stoveproduction_ef_ch4_kerosene_project'][i]
    stoveproduction_ef_n2O_kerosene_project=df['stoveproduction_ef_n2O_kerosene_project'][i]


    stoveproduction_ef_co2_LPG_project=df['stoveproduction_ef_co2_LPG_project'][i]
    stoveproduction_ef_ch4_LPG_project=df['stoveproduction_ef_ch4_LPG_project'][i]
    stoveproduction_ef_n2O_LPG_project=df['stoveproduction_ef_n2O_LPG_project'][i]


    stoveproduction_ef_co2_ethanol_project=df['stoveproduction_ef_co2_ethanol_project'][i]
    stoveproduction_ef_ch4_ethanol_project=df['stoveproduction_ef_ch4_ethanol_project'][i]
    stoveproduction_ef_n2O_ethanol_project=df['stoveproduction_ef_n2O_ethanol_project'][i]


    stoveproduction_ef_co2_electric_project=df['stoveproduction_ef_co2_electric_project'][i]
    stoveproduction_ef_ch4_electric_project=df['stoveproduction_ef_ch4_electric_project'][i]
    stoveproduction_ef_n2O_electric_project=df['stoveproduction_ef_n2O_electric_project'][i]


    stoveproduction_ef_co2_pellets_project=df['stoveproduction_ef_co2_pellets_project'][i]
    stoveproduction_ef_ch4_pellets_project=df['stoveproduction_ef_ch4_pellets_project'][i]
    stoveproduction_ef_n2O_pellets_project=df['stoveproduction_ef_n2O_pellets_project'][i]

    # Project Feedstock Emission Factors

    feedstockproduction_ef_co2_firewood_project=df['feedstockproduction_ef_co2_firewood_project'][i]
    feedstockproduction_ef_ch4_firewood_project=df['feedstockproduction_ef_ch4_firewood_project'][i]
    feedstockproduction_ef_n2O_firewood_project=df['feedstockproduction_ef_n2O_firewood_project'][i]


    feedstockproduction_ef_co2_charcoal_project=df['feedstockproduction_ef_co2_charcoal_project'][i]
    feedstockproduction_ef_ch4_charcoal_project=df['feedstockproduction_ef_ch4_charcoal_project'][i]
    feedstockproduction_ef_n2O_charcoal_project=df['feedstockproduction_ef_n2O_charcoal_project'][i]


    feedstockproduction_ef_co2_kerosene_project=df['feedstockproduction_ef_co2_kerosene_project'][i]
    feedstockproduction_ef_ch4_kerosene_project=df['feedstockproduction_ef_ch4_kerosene_project'][i]
    feedstockproduction_ef_n2O_kerosene_project=df['feedstockproduction_ef_n2O_kerosene_project'][i]


    feedstockproduction_ef_co2_LPG_project=df['feedstockproduction_ef_co2_LPG_project'][i]
    feedstockproduction_ef_ch4_LPG_project=df['feedstockproduction_ef_ch4_LPG_project'][i]
    feedstockproduction_ef_n2O_LPG_project=df['feedstockproduction_ef_n2O_LPG_project'][i]


    feedstockproduction_ef_co2_ethanol_project=df['feedstockproduction_ef_co2_ethanol_project'][i]
    feedstockproduction_ef_ch4_ethanol_project=df['feedstockproduction_ef_ch4_ethanol_project'][i]
    feedstockproduction_ef_n2O_ethanol_project=df['feedstockproduction_ef_n2O_ethanol_project'][i]


    feedstockproduction_ef_co2_electric_project=df['feedstockproduction_ef_co2_electric_project'][i]
    feedstockproduction_ef_ch4_electric_project=df['feedstockproduction_ef_ch4_electric_project'][i]
    feedstockproduction_ef_n2O_electric_project=df['feedstockproduction_ef_n2O_electric_project'][i]


    feedstockproduction_ef_co2_pellets_project=df['feedstockproduction_ef_co2_pellets_project'][i]
    feedstockproduction_ef_ch4_pellets_project=df['feedstockproduction_ef_ch4_pellets_project'][i]
    feedstockproduction_ef_n2O_pellets_project=df['feedstockproduction_ef_n2O_pellets_project'][i]


    # Project Distribution Emission Factors
    distribution_ef_co2_firewood_project=df['distribution_ef_co2_firewood_project'][i]
    distribution_ef_ch4_firewood_project=df['distribution_ef_ch4_firewood_project'][i]
    distribution_ef_n2O_firewood_project=df['distribution_ef_n2O_firewood_project'][i]


    distribution_ef_co2_charcoal_project=df['distribution_ef_co2_charcoal_project'][i]
    distribution_ef_ch4_charcoal_project=df['distribution_ef_ch4_charcoal_project'][i]
    distribution_ef_n2O_charcoal_project=df['distribution_ef_n2O_charcoal_project'][i]


    distribution_ef_co2_kerosene_project=df['distribution_ef_co2_kerosene_project'][i]
    distribution_ef_ch4_kerosene_project=df['distribution_ef_ch4_kerosene_project'][i]
    distribution_ef_n2O_kerosene_project=df['distribution_ef_n2O_kerosene_project'][i]


    distribution_ef_co2_LPG_project=df['distribution_ef_co2_LPG_project'][i]
    distribution_ef_ch4_LPG_project=df['distribution_ef_ch4_LPG_project'][i]
    distribution_ef_n2O_LPG_project=df['distribution_ef_n2O_LPG_project'][i]


    distribution_ef_co2_ethanol_project=df['distribution_ef_co2_ethanol_project'][i]
    distribution_ef_ch4_ethanol_project=df['distribution_ef_ch4_ethanol_project'][i]
    distribution_ef_n2O_ethanol_project=df['distribution_ef_n2O_ethanol_project'][i]


    distribution_ef_co2_electric_project=df['distribution_ef_co2_electric_project'][i]
    distribution_ef_ch4_electric_project=df['distribution_ef_ch4_electric_project'][i]
    distribution_ef_n2O_electric_project=df['distribution_ef_n2O_electric_project'][i]


    distribution_ef_co2_pellets_project=df['distribution_ef_co2_pellets_project'][i]
    distribution_ef_ch4_pellets_project=df['distribution_ef_ch4_pellets_project'][i]
    distribution_ef_n2O_pellets_project=df['distribution_ef_n2O_pellets_project'][i]

    # Project Processing Emission Factors
    processing_ef_co2_firewood_project=df['processing_ef_co2_firewood_project'][i]
    processing_ef_ch4_firewood_project=df['processing_ef_ch4_firewood_project'][i]
    processing_ef_n2O_firewood_project=df['processing_ef_n2O_firewood_project'][i]


    processing_ef_co2_charcoal_project=df['processing_ef_co2_charcoal_project'][i]
    processing_ef_ch4_charcoal_project=df['processing_ef_ch4_charcoal_project'][i]
    processing_ef_n2O_charcoal_project=df['processing_ef_n2O_charcoal_project'][i]


    processing_ef_co2_kerosene_project=df['processing_ef_co2_kerosene_project'][i]
    processing_ef_ch4_kerosene_project=df['processing_ef_ch4_kerosene_project'][i]
    processing_ef_n2O_kerosene_project=df['processing_ef_n2O_kerosene_project'][i]


    processing_ef_co2_LPG_project=df['processing_ef_co2_LPG_project'][i]
    processing_ef_ch4_LPG_project=df['processing_ef_ch4_LPG_project'][i]
    processing_ef_n2O_LPG_project=df['processing_ef_n2O_LPG_project'][i]


    processing_ef_co2_ethanol_project=df['processing_ef_co2_ethanol_project'][i]
    processing_ef_ch4_ethanol_project=df['processing_ef_ch4_ethanol_project'][i]
    processing_ef_n2O_ethanol_project=df['processing_ef_n2O_ethanol_project'][i]


    processing_ef_co2_electric_project=df['processing_ef_co2_electric_project'][i]
    processing_ef_ch4_electric_project=df['processing_ef_ch4_electric_project'][i]
    processing_ef_n2O_electric_project=df['processing_ef_n2O_electric_project'][i]


    processing_ef_co2_pellets_project=df['processing_ef_co2_pellets_project'][i]
    processing_ef_ch4_pellets_project=df['processing_ef_ch4_pellets_project'][i]
    processing_ef_n2O_pellets_project=df['processing_ef_n2O_pellets_project'][i]


    #fNRB
    mofus_fNRB_bailis=df['mofus_fNRB_bailis'][i]
    min_mofus_fNRB_bailis=mofus_fNRB_bailis*0.9
    max_mofus_fNRB_bailis=mofus_fNRB_bailis*1.1


    n_fNRB=np.random.triangular(left=min_mofus_fNRB_bailis, mode=mofus_fNRB_bailis, right=max_mofus_fNRB_bailis, size=(N))

    # Adoption, Usage, and Leakage

    #adoption_definition

    project_adoption=df['project_adoption'][i]


    project_usage=df['project_usage'][i]

    kpt_adj_usage=np.random.uniform(0,0.53,N)

    kpt_usage=project_usage*(1-kpt_adj_usage)


    project_adoption_commercial=df['project_adoption_commercial'][i]


    project_usage_commercial=df['project_usage_commercial'][i]

    project_stacking=df['project_stacking'][i]

    project_stacking_commercial=df['project_stacking_commercial'][i]

    kpt_adj_stacking=np.random.uniform(0,0.29,N)

    kpt_stacking=project_stacking*kpt_adj_stacking

    leakage_binary=df['leakage_binary'][i]



    leakage_default= df['leakage_default'][i] # put 1 if not being multiplied



    leakage_tCO2e=df['leakage_tCO2e'][i]



    more_than_one_stove_adj=df['more_than_one_stove_adj'][i]

    other_tCO2e_reduction=df['other_tCO2e_reduction'][i]

    #rebound=1

    rebound=df['rebound'][i]



    # Baseline Emissions
    # Baseline Firewood

    be_firewood_gillwiehl=prc_firewood_baseline*(ncv_firewood_agw*kg_firewood_baseline/1000*((n_fNRB*(ef_combustion_co2_firewood
                                                                                                      +ef_combustion_ch4_firewood*gwp_ch4_agw
                                                                                                      + ef_combustion_n2O_firewood*gwp_n2O_agw)
                                                                                              # non-renewable
                                                                                              +(1-n_fNRB)*(ef_combustion_ch4_firewood*gwp_ch4_renewable
                                                                                                      + ef_combustion_n2O_firewood*gwp_n2O_agw))))
                                                                                             # renewable emissions use a different ch4 gwp

    # Baseline Charcoal

    be_charcoal_gillwiehl=prc_charcoal_baseline*(ncv_charcoal_agw*kg_charcoal_baseline/1000*((n_fNRB*(ef_combustion_co2_charcoal
                                                                                                      +ef_combustion_ch4_charcoal*gwp_ch4_agw
                                                                                                      + ef_combustion_n2O_charcoal*gwp_n2O_agw)
                                                                                              # non-renewable
                                                                                              +(1-n_fNRB)*(ef_combustion_ch4_charcoal*gwp_ch4_renewable
                                                                                                      + ef_combustion_n2O_charcoal*gwp_n2O_agw))
                                                                                             # renewable emissions use a different ch4 gwp


                                                                                             +ef_total_not_combustion_co2_charcoal
                                                                                             +ef_total_not_combustion_ch4_charcoal*gwp_ch4_agw
                                                                                             +ef_total_not_combustion_n2O_charcoal*gwp_n2O_agw))
    # Baseline LPG

    be_LPG_gillwiehl=prc_lpg_baseline*(ncv_LPG_agw*kg_lpg_baseline/1000*(ef_total_co2_LPG +ef_total_ch4_LPG*gwp_ch4_agw
                                                                                             +ef_total_n2O_LPG*gwp_n2O_agw))
    # Baseline Kerosene

    be_kerosene_gillwiehl=prc_kerosene_baseline*(ncv_kerosene_agw*kg_kerosene_baseline/1000*(ef_total_co2_kerosene +ef_total_ch4_kerosene*gwp_ch4_agw
                                                                                             +ef_total_n2O_kerosene*gwp_n2O_agw))
    # Baseline Ethanol

    be_ethanol_gillwiehl=prc_ethanol_baseline*(ncv_ethanol_agw*l_ethanol_baseline_commercial*kg_per_liter_ethanol_baseline/1000*(ef_total_zeroed_co2_ethanol +ef_total_zeroed_ch4_ethanol*gwp_ch4_agw
                                                                                             +ef_total_zeroed_n2O_ethanol*gwp_n2O_agw))
    # Baseline Pellets

    be_pellets_gillwiehl=prc_pellets_baseline*(ncv_pellets_agw*kg_pellets_baseline/1000*(ef_total_zeroed_co2_pellets +ef_total_zeroed_ch4_pellets*gwp_ch4_agw
                                                                                             +ef_total_zeroed_n2O_pellets*gwp_n2O_agw))
    # Total Baseline
    be_total_domestic_gillwiehl = be_firewood_gillwiehl + be_charcoal_gillwiehl + be_LPG_gillwiehl+ be_ethanol_gillwiehl + be_pellets_gillwiehl

    be_total_domestic_gillwiehl

   # Baseline Commercial Emissions
    # Baseline Commercial Firewood

    be_firewood_commercial_gillwiehl=prc_firewood_baseline_commercial*(ncv_firewood_agw*kg_firewood_baseline_commercial/1000*((n_fNRB*(ef_combustion_co2_firewood
                                                                                                      +ef_combustion_ch4_firewood*gwp_ch4_agw
                                                                                                      + ef_combustion_n2O_firewood*gwp_n2O_agw)
                                                                                              # non-renewable
                                                                                              +(1-n_fNRB)*(ef_combustion_ch4_firewood*gwp_ch4_renewable
                                                                                                      + ef_combustion_n2O_firewood*gwp_n2O_agw))))
                                                                                             # renewable emissions use a different ch4 gwp

    # Baseline Charcoal

    be_charcoal_commercial_gillwiehl=prc_charcoal_baseline_commercial*(ncv_charcoal_agw*kg_charcoal_baseline_commercial/1000*((n_fNRB*(ef_combustion_co2_charcoal
                                                                                                      +ef_combustion_ch4_charcoal*gwp_ch4_agw
                                                                                                      + ef_combustion_n2O_charcoal*gwp_n2O_agw)
                                                                                              # non-renewable
                                                                                              +(1-n_fNRB)*(ef_combustion_ch4_charcoal*gwp_ch4_renewable
                                                                                                      + ef_combustion_n2O_charcoal*gwp_n2O_agw))
                                                                                             # renewable emissions use a different ch4 gwp


                                                                                             +ef_total_not_combustion_co2_charcoal
                                                                                             +ef_total_not_combustion_ch4_charcoal*gwp_ch4_agw
                                                                                             +ef_total_not_combustion_n2O_charcoal*gwp_n2O_agw))
    # Baseline LPG

    be_LPG_commercial_gillwiehl=prc_lpg_baseline_commercial*(ncv_LPG_agw*kg_lpg_baseline_commercial/1000*(ef_total_co2_LPG +ef_total_ch4_LPG*gwp_ch4_agw
                                                                                             +ef_total_n2O_LPG*gwp_n2O_agw))
    # Baseline Kerosene

    be_kerosene_commercial_gillwiehl=prc_kerosene_baseline*(ncv_kerosene_agw*kg_kerosene_baseline_commercial/1000*(ef_total_co2_kerosene +ef_total_ch4_kerosene*gwp_ch4_agw
                                                                                             +ef_total_n2O_kerosene*gwp_n2O_agw))
    # Baseline Ethanol

    be_ethanol_commercial_gillwiehl=prc_ethanol_baseline*(ncv_ethanol_agw*l_ethanol_baseline_commercial*kg_per_liter_ethanol_baseline/1000*(ef_total_zeroed_co2_ethanol +ef_total_zeroed_ch4_ethanol*gwp_ch4_agw
                                                                                             +ef_total_zeroed_n2O_ethanol*gwp_n2O_agw))
    # Baseline Ethanol

    be_pellets_commercial_gillwiehl=prc_pellets_baseline*(ncv_pellets_agw*kg_pellets_baseline/1000*(ef_total_zeroed_co2_pellets +ef_total_zeroed_ch4_pellets*gwp_ch4_agw
                                                                                             +ef_total_zeroed_n2O_pellets*gwp_n2O_agw))
    # Total Baseline
    be_total_commercial_gillwiehl = be_firewood_commercial_gillwiehl + be_charcoal_commercial_gillwiehl + be_LPG_commercial_gillwiehl+ be_ethanol_commercial_gillwiehl + be_pellets_commercial_gillwiehl

    be_total_commercial_gillwiehl

    # Project Emissions
    # Project Firewood

    pe_firewood_gillwiehl=prc_firewood_project*(ncv_firewood_agw*kg_firewood_project/1000*((n_fNRB*(ef_combustion_co2_firewood
                                                                                                      +ef_combustion_ch4_firewood*gwp_ch4_agw
                                                                                                      + ef_combustion_n2O_firewood*gwp_n2O_agw)
                                                                                              # non-renewable
                                                                                              +(1-n_fNRB)*(ef_combustion_ch4_firewood*gwp_ch4_renewable
                                                                                                      + ef_combustion_n2O_firewood*gwp_n2O_agw))))
                                                                                             # renewable emissions use a different ch4 gwp

    # Project Charcoal

    pe_charcoal_gillwiehl=prc_charcoal_project*(ncv_charcoal_agw*kg_charcoal_project/1000*((n_fNRB*(ef_combustion_co2_charcoal
                                                                                                      +ef_combustion_ch4_charcoal*gwp_ch4_agw
                                                                                                      + ef_combustion_n2O_charcoal*gwp_n2O_agw)
                                                                                              # non-renewable
                                                                                              +(1-n_fNRB)*(ef_combustion_ch4_charcoal*gwp_ch4_renewable
                                                                                                      + ef_combustion_n2O_charcoal*gwp_n2O_agw))
                                                                                             # renewable emissions use a different ch4 gwp


                                                                                             +ef_total_not_combustion_co2_charcoal
                                                                                             +ef_total_not_combustion_ch4_charcoal*gwp_ch4_agw
                                                                                             +ef_total_not_combustion_n2O_charcoal*gwp_n2O_agw))
    # Project LPG

    pe_LPG_gillwiehl=prc_lpg_project*(ncv_LPG_agw*kg_lpg_project/1000*(ef_total_co2_LPG +ef_total_ch4_LPG*gwp_ch4_agw
                                                                                             +ef_total_n2O_LPG*gwp_n2O_agw))
    # Project Kerosene

    pe_kerosene_gillwiehl=prc_kerosene_project*(ncv_kerosene_agw*kg_kerosene_project/1000*(ef_total_co2_kerosene +ef_total_ch4_kerosene*gwp_ch4_agw
                                                                                             +ef_total_n2O_kerosene*gwp_n2O_agw))
    # Project Ethanol

    pe_ethanol_gillwiehl=prc_ethanol_project*(ncv_ethanol_agw*l_ethanol_project*kg_per_liter_ethanol_project/1000*(ef_total_zeroed_co2_ethanol +ef_total_zeroed_ch4_ethanol*gwp_ch4_agw
                                                                                             +ef_total_zeroed_n2O_ethanol*gwp_n2O_agw))
    # Project Pellets

    pe_pellets_gillwiehl=prc_pellets_project*(ncv_pellets_agw*kg_pellets_project/1000*(ef_total_zeroed_co2_pellets +ef_total_zeroed_ch4_pellets*gwp_ch4_agw
                                                                                             +ef_total_zeroed_n2O_pellets*gwp_n2O_agw))
    # Total project
    pe_total_domestic_gillwiehl = pe_firewood_gillwiehl + pe_charcoal_gillwiehl + pe_LPG_gillwiehl+ pe_ethanol_gillwiehl + pe_pellets_gillwiehl

    pe_total_domestic_gillwiehl

    # Project Commercial Emissions
    # Project Commercial Firewood

    pe_firewood_commercial_gillwiehl=prc_firewood_project_commercial*(ncv_firewood_agw*kg_firewood_project_commercial/1000*((n_fNRB*(ef_combustion_co2_firewood
                                                                                                      +ef_combustion_ch4_firewood*gwp_ch4_agw
                                                                                                      + ef_combustion_n2O_firewood*gwp_n2O_agw)
                                                                                              # non-renewable
                                                                                              +(1-n_fNRB)*(ef_combustion_ch4_firewood*gwp_ch4_renewable
                                                                                                      + ef_combustion_n2O_firewood*gwp_n2O_agw))))
                                                                                             # renewable emissions use a different ch4 gwp

    # Project Charcoal

    pe_charcoal_commercial_gillwiehl=prc_charcoal_project_commercial*(ncv_charcoal_agw*kg_charcoal_project_commercial/1000*((n_fNRB*(ef_combustion_co2_charcoal
                                                                                                      +ef_combustion_ch4_charcoal*gwp_ch4_agw
                                                                                                      + ef_combustion_n2O_charcoal*gwp_n2O_agw)
                                                                                              # non-renewable
                                                                                              +(1-n_fNRB)*(ef_combustion_ch4_charcoal*gwp_ch4_renewable
                                                                                                      + ef_combustion_n2O_charcoal*gwp_n2O_agw))
                                                                                             # renewable emissions use a different ch4 gwp


                                                                                             +ef_total_not_combustion_co2_charcoal
                                                                                             +ef_total_not_combustion_ch4_charcoal*gwp_ch4_agw
                                                                                             +ef_total_not_combustion_n2O_charcoal*gwp_n2O_agw))
    # Project LPG

    pe_LPG_commercial_gillwiehl=prc_lpg_project_commercial*(ncv_LPG_agw*kg_lpg_project_commercial/1000*(ef_total_co2_LPG +ef_total_ch4_LPG*gwp_ch4_agw
                                                                                             +ef_total_n2O_LPG*gwp_n2O_agw))
    # Project Kerosene

    pe_kerosene_commercial_gillwiehl=prc_kerosene_project_commercial*(ncv_kerosene_agw*kg_kerosene_project_commercial/1000*(ef_total_co2_kerosene +ef_total_ch4_kerosene*gwp_ch4_agw
                                                                                             +ef_total_n2O_kerosene*gwp_n2O_agw))
    # Project Ethanol

    pe_ethanol_commercial_gillwiehl=prc_ethanol_project_commercial*(ncv_ethanol_agw*l_ethanol_project_commercial*kg_per_liter_ethanol_project/1000*(ef_total_zeroed_co2_ethanol +ef_total_zeroed_ch4_ethanol*gwp_ch4_agw
                                                                                             +ef_total_zeroed_n2O_ethanol*gwp_n2O_agw))
    # Project Ethanol

    pe_pellets_commercial_gillwiehl=prc_pellets_project_commercial*(ncv_pellets_agw*kg_pellets_project_commercial/1000*(ef_total_zeroed_co2_pellets +ef_total_zeroed_ch4_pellets*gwp_ch4_agw
                                                                                             +ef_total_zeroed_n2O_pellets*gwp_n2O_agw))
    # Total project
    pe_total_commercial_gillwiehl = pe_firewood_commercial_gillwiehl + pe_charcoal_commercial_gillwiehl + pe_LPG_commercial_gillwiehl+ pe_ethanol_commercial_gillwiehl + pe_pellets_commercial_gillwiehl

    pe_total_commercial_gillwiehl




    ## Adjusting adoption, usage, and stacking for the different methodologies



    be_total_agw=be_total_domestic_gillwiehl

    be_total_agw=be_total_agw+be_total_commercial_gillwiehl

    pe_total_agw=pe_total_domestic_gillwiehl
    pe_total_agw=pe_total_agw+pe_total_commercial_gillwiehl

    #be_total_agw= be_total_agw*n_adoption*n_usage*n_stacking

    #pe_total_agw= be_total_agw*n_adoption*n_usage*n_stacking

    if df.protocol_type[i]=='GS Metered'and df.stove_year_cohort[i]!='commercial':

        be_total_agw= be_total_agw*project_adoption*project_usage*(1-project_stacking)

        pe_total_agw= pe_total_agw*project_adoption*project_usage*(1-project_stacking)

    if df.protocol_type[i]=='GS TPDDTEC'and df.protocol_id[i]!='GS2758'and df.stove_year_cohort[i]!='commercial':

        be_total_agw= be_total_agw*n_adoption*kpt_usage*(1-kpt_stacking)

        pe_total_agw= pe_total_agw*n_adoption*kpt_usage*(1-kpt_stacking)

    if df.protocol_type[i]=='GS TPDDTEC'and df.stove_year_cohort[i]=='commercial':

        be_total_agw= be_total_agw*project_adoption*project_usage*(1-project_stacking)

        pe_total_agw= pe_total_agw*project_adoption*project_usage*(1-project_stacking)


    if df.protocol_type[i]=='GS Simplified'and df.stove_year_cohort[i]!='commercial':

        be_total_agw= be_total_agw*n_adoption*n_usage*(1-n_stacking)

        pe_total_agw= pe_total_agw*n_adoption*n_usage*(1-n_stacking)

    if df.protocol_type[i]=='CDM AMS II G'and df.protocol_id[i]!='GS10974'and df.stove_year_cohort[i]!='commercial':


        be_total_agw= be_total_agw*n_adoption*n_usage*(1-n_stacking)

        pe_total_agw= pe_total_agw*n_adoption*n_usage*(1-n_stacking)

    if df.protocol_type[i]=='CDM AMS I E'and df.stove_year_cohort[i]!='commercial':

        be_total_agw= be_total_agw*n_adoption*n_usage*(1-project_stacking)

        pe_total_agw= pe_total_agw*n_adoption*n_usage*(1-project_stacking)

    if df.protocol_id[i]=='GS2758':

        be_total_agw= be_total_agw*req_adoption*kpt_usage*(1-kpt_stacking)

        pe_total_agw= pe_total_agw*req_adoption*kpt_usage*(1-kpt_stacking)


    if df.protocol_id[i]=='GS10974'and df.stove_year_cohort[i]!='commercial':

        be_total_agw= be_total_agw*req_adoption*req_usage*(1-req_stacking)

        pe_total_agw= pe_total_agw*req_adoption*req_usage*(1-req_stacking)


    if df.protocol_type[i]=='CDM AMS I E'and df.stove_year_cohort[i]=='commercial':

        be_total_agw= be_total_agw*project_adoption*project_usage*(1-project_stacking)

        pe_total_agw= pe_total_agw*project_adoption*project_usage*(1-project_stacking)


    if df.protocol_type[i]=='CDM AMS II G'and df.stove_year_cohort[i]=='commercial':

        be_total_agw= be_total_agw*project_adoption*project_usage*(1-project_stacking)

        pe_total_agw= pe_total_agw*project_adoption*project_usage*(1-project_stacking)


    # Leakage if not default
    le_total_agw=(leakage_tCO2e+other_tCO2e_reduction)/(n_devices_domestic+n_devices_commercial)/(project_days+project_days_commercial) # Project Emission Reductions (PERs) in (tCO2e)





    # Totally for all
    PEs_agw=(pe_total_agw)# Project Emission Reductions (PERs) in (tCO2e)
    BEs_agw=(be_total_agw)# Project Emission Reductions (PERs) in (tCO2e)

    VERs_agw=rebound*(be_total_agw-pe_total_agw-le_total_agw)*more_than_one_stove_adj*leakage_default


    # Protocol Analysis:


    #All

    all_VERS_agw.append(VERs_agw)

    sum_VERS_not_per_stove_per_day_agw=sum_VERS_not_per_stove_per_day_agw+VERs_agw*n_devices_domestic*project_days



    #bangladesh
    if df.country[i]=='Bangladesh':
        bangladesh_N=bangladesh_N+1
        bangladesh_VERs_agw.append(VERs_agw)

    #burkina faso
    if df.country[i]=='Burkina Faso':
        burkinafaso_N=burkinafaso_N+1
        burkinafaso_VERs_agw.append(VERs_agw)

    #cameroon
    if df.country[i]=='Cameroon':
        cameroon_N=cameroon_N+1
        cameroon_VERs_agw.append(VERs_agw)

    #china
    if df.country[i]=='China':
        china_N=china_N+1
        china_VERs_agw.append(VERs_agw)

    #ethiopia
    if df.country[i]=='Ethiopia':
        ethiopia_N=ethiopia_N+1
        ethiopia_VERs_agw.append(VERs_agw)

    #ghana
    if df.country[i]=='Ghana':
        ghana_N=ghana_N+1
        ghana_VERs_agw.append(VERs_agw)

    #guatemala
    if df.country[i]=='Guatemala':
        guatemala_N=guatemala_N+1
        guatemala_VERs_agw.append(VERs_agw)

    #haiti
    if df.country[i]=='Haiti':
        haiti_N=haiti_N+1
        haiti_VERs_agw.append(VERs_agw)

    #honduras
    if df.country[i]=='Honduras':
        honduras_N=honduras_N+1
        honduras_VERs_agw.append(VERs_agw)

    #india
    if df.country[i]=='India':
        india_N=india_N+1
        india_VERs_agw.append(VERs_agw)

    #kenya
    if df.country[i]=='Kenya':
        kenya_N=kenya_N+1
        kenya_VERs_agw.append(VERs_agw)

    #lesotho
    if df.country[i]=='Lesotho':
        lesotho_N=lesotho_N+1
        lesotho_VERs_agw.append(VERs_agw)

    #malawi
    if df.country[i]=='Malawi':
        malawi_N=malawi_N+1
        malawi_VERs_agw.append(VERs_agw)

    #mali
    if df.country[i]=='Mali':
        mali_N=mali_N+1
        mali_VERs_agw.append(VERs_agw)

    #mexico
    if df.country[i]=='Mexico':
        mexico_N=mexico_N+1
        mexico_VERs_agw.append(VERs_agw)

    #mozambique
    if df.country[i]=='Mozambique':
        mozambique_N=mozambique_N+1
        mozambique_VERs_agw.append(VERs_agw)

    #myanmar
    if df.country[i]=='Myanmar':
        myanmar_N=myanmar_N+1
        myanmar_VERs_agw.append(VERs_agw)

    #nepal
    if df.country[i]=='Nepal':
        nepal_N=nepal_N+1
        nepal_VERs_agw.append(VERs_agw)

    #nigeria
    if df.country[i]=='Nigeria':
        nigeria_N=nigeria_N+1
        nigeria_VERs_agw.append(VERs_agw)

    #peru
    if df.country[i]=='Peru':
        peru_N=peru_N+1
        peru_VERs_agw.append(VERs_agw)

    #rwanda
    if df.country[i]=='Rwanda':
        rwanda_N=rwanda_N+1
        rwanda_VERs_agw.append(VERs_agw)

    #sudan
    if df.country[i]=='Sudan':
        sudan_N=sudan_N+1
        sudan_VERs_agw.append(VERs_agw)

    #uganda
    if df.country[i]=='Uganda':
        uganda_N=uganda_N+1
        uganda_VERs_agw.append(VERs_agw)

    #zambia
    if df.country[i]=='Zambia':
        zambia_N=zambia_N+1
        zambia_VERs_agw.append(VERs_agw)

    #gs metered pellets
    if df.protocol_type[i]=='GS Metered':
        GS_Meter_N=GS_Meter_N+1
        GS_Meter_VERs_agw.append(VERs_agw)

    #gs_lpg
    if df.stove[i]=='gs_lpg':
        gs_lpg_N=gs_lpg_N+1
        gs_lpg_VERs_agw.append(VERs_agw)

    #gs_imp_firewood
    if df.stove[i]=='gs_imp_firewood':
        gs_imp_firewood_N=gs_imp_firewood_N+1
        gs_imp_firewood_VERs_agw.append(VERs_agw)

    #gs_simple_firewood
    if df.stove[i]=='gs_simple_firewood':
        gs_simple_firewood_N=gs_simple_firewood_N+1
        gs_simple_firewood_VERs_agw.append(VERs_agw)


    #gs_imp_charcoal
    if df.stove[i]=='gs_imp_charcoal':
        gs_imp_charcoal_N=gs_imp_charcoal_N+1
        gs_imp_charcoal_VERs_agw.append(VERs_agw)

    #cdm_imp_firewood
    if df.stove[i]=='cdm_imp_firewood':
        cdm_imp_firewood_N=cdm_imp_firewood_N+1
        cdm_imp_firewood_VERs_agw.append(VERs_agw)
        print(np.mean(VERs_agw))


    #cdm_imp_charcoal
    if df.stove[i]=='cdm_imp_charcoal':
        cdm_imp_charcoal_N=cdm_imp_charcoal_N+1
        cdm_imp_charcoal_VERs_agw.append(VERs_agw)

    # CDM I E
    if df.protocol_type[i]=='CDM AMS I E':
        CDM_I_E_N=CDM_I_E_N+1
        CDM_I_E_VERs_agw.append(VERs_agw)


    #Times Over Credited
    ratio_analysis_project=VERs_agw/VERs
    ratio_analysis_project_mean=np.mean(VERs_agw)/VERs*100
    ratio_analysis_project_mean
    ratio_analysis_project_mean_compilation.append(ratio_analysis_project_mean)


    ratio_project_analysis=VERs/VERs_agw
    ratio_project_analysis_mean=VERs/np.mean(VERs_agw)
    ratio_project_analysis_mean
    ratio_project_analysis_mean_compilation.append(ratio_project_analysis_mean)
    z=1.96

    sd_VERs_agw=np.std(VERs_agw)
    sd_ratio_analysis_project=np.std(ratio_analysis_project)
    sd_ratio_project_analysis=np.std(ratio_project_analysis)


    ratio_analysis_project_lower=ratio_analysis_project_mean - z*sd_ratio_analysis_project/np.sqrt(N)
    ratio_analysis_project_upper=ratio_analysis_project_mean + z*sd_ratio_analysis_project/np.sqrt(N)

    ratio_analysis_project_lower_compilation.append(ratio_analysis_project_lower)
    ratio_analysis_project_upper_compilation.append(ratio_analysis_project_upper)


    ratio_project_analysis_lower=ratio_project_analysis_mean - z*sd_ratio_project_analysis/np.sqrt(N)
    ratio_project_analysis_upper=ratio_project_analysis_mean + z*sd_ratio_project_analysis/np.sqrt(N)

    #print(sd_ratio_project_analysis)

    ratio_project_analysis_lower_compilation.append(ratio_project_analysis_lower)
    ratio_project_analysis_upper_compilation.append(ratio_project_analysis_upper)

    #if ratio_project_analysis_mean<2:
    print(np.mean(VERs_agw))
    print(VERs)
    print('We find that', protocol_id,  'project reports emissions which are',round(ratio_project_analysis_mean,2),'[',round(ratio_project_analysis_lower,2),' , ',round(ratio_project_analysis_upper,2),']','times overcredited.')
    print('Our analysis finds emissions which are',round(ratio_analysis_project_mean,2),'[',round(ratio_analysis_project_lower,2),' , ',round(ratio_analysis_project_upper,2),']','% of the verified credits in the PDD.')

    i=i+1
    ALL_N=ALL_N+1

0.00022081554846381803
0.00022081554846381803
0.003653365024898631
We find that PoA 10471 CPA1 project reports emissions which are 16.54 [ 16.02  ,  17.07 ] times overcredited.
Our analysis finds emissions which are 6.04 [ 6.04  ,  6.04 ] % of the verified credits in the PDD.
0.00022064242898995655
0.00022064242898995655
0.003653365024898631
We find that PoA 10471 CPA2 project reports emissions which are 16.56 [ 16.04  ,  17.08 ] times overcredited.
Our analysis finds emissions which are 6.04 [ 6.04  ,  6.04 ] % of the verified credits in the PDD.
0.0002206793133570503
0.0002206793133570503
0.003653365024898631
We find that PoA 10471 CPA3 project reports emissions which are 16.56 [ 16.03  ,  17.08 ] times overcredited.
Our analysis finds emissions which are 6.04 [ 6.04  ,  6.04 ] % of the verified credits in the PDD.
0.0002207651160059496
0.0002207651160059496
0.003653365024898631
We find that PoA 10471 CPA4 project reports emissions which are 16.55 [ 16.03  ,  17.07 ] times overcredit

In [10]:
ratio_project_analysis_mean_compilation=pd.DataFrame(ratio_project_analysis_mean_compilation)
ratio_project_analysis_mean_compilation.to_excel('ratio_project_analysis_mean_compilation.xlsx')

ratio_project_analysis_lower_compilation=pd.DataFrame(ratio_project_analysis_lower_compilation)
ratio_project_analysis_lower_compilation.to_excel('ratio_project_analysis_lower_compilation.xlsx')

ratio_project_analysis_upper_compilation=pd.DataFrame(ratio_project_analysis_upper_compilation)
ratio_project_analysis_upper_compilation.to_excel('ratio_project_analysis_upper_compilation.xlsx')

In [11]:
ratio_analysis_project_mean_compilation=pd.DataFrame(ratio_analysis_project_mean_compilation)
ratio_analysis_project_mean_compilation.to_excel('ratio_analysis_project_mean_compilation.xlsx')

ratio_analysis_project_lower_compilation=pd.DataFrame(ratio_analysis_project_lower_compilation)
ratio_analysis_project_lower_compilation.to_excel('ratio_analysis_project_lower_compilation.xlsx')

ratio_analysis_project_upper_compilation=pd.DataFrame(ratio_analysis_project_upper_compilation)
ratio_analysis_project_upper_compilation.to_excel('ratio_analysis_project_upper_compilation.xlsx')

In [12]:
def weighted_average(dataframe, value, weight):
    val = dataframe[value]
    wt = dataframe[weight]
    return (val * wt).sum() / wt.sum()

In [13]:
def weighted_average_grouped(dataframe, group_var, value, weight):
    dataframe_grouped=dataframe.groupby(group_var)
    val = dataframe_grouped[value]
    wt = dataframe_grouped[weight]
    return (val * wt).sum() / wt.sum()

In [14]:
PoAs=pd.DataFrame(df['PoA'])
ids=pd.DataFrame(df['protocol_id'])
category=pd.DataFrame(df['stove'])
country=pd.DataFrame(df['country'])
total_credits=pd.DataFrame(df['total_verified_credits'])
stove_days=pd.DataFrame(df['stove_days'])

df_info=PoAs.join(ids)
df_info=df_info.join(category)
df_info=df_info.join(country)
df_info=df_info.join(total_credits)
df_info=df_info.join(stove_days)
df_info=pd.DataFrame(df_info)
df_info=pd.concat([df_info[1:].reset_index(drop=True),ratio_project_analysis_lower_compilation], axis=1)
df_info= pd.concat([df_info.reset_index(drop=True),ratio_project_analysis_mean_compilation], axis=1)
df_info= pd.concat([df_info.reset_index(drop=True),ratio_project_analysis_upper_compilation], axis=1)
df_info.columns = ['PoA','protocol_id', 'stove', 'country','total_verified_credits','stove_days','lower','mean','upper']
df_info['stove_days_mean_overcrediting']=df_info['mean']*df_info['stove_days']
df_info
df_project_weighted_stove_days_numerator=df_info.groupby('protocol_id')['stove_days_mean_overcrediting'].sum()
df_project_weighted_stove_days_numerator=pd.DataFrame(df_project_weighted_stove_days_numerator)


df_project_weighted_stove_days_denominator=df_info.groupby('protocol_id')['stove_days'].sum()
df_project_weighted_stove_days_denominator=pd.DataFrame(df_project_weighted_stove_days_denominator)
df_project_weighted_stove_days=df_project_weighted_stove_days_numerator['stove_days_mean_overcrediting']/df_project_weighted_stove_days_denominator['stove_days']
df_project_weighted_stove_days=pd.DataFrame(df_project_weighted_stove_days)
df_project_weighted_stove_days.mean() #This is the average project over-crediting
print(df_project_weighted_stove_days.mean())
df_project_weighted_stove_days
protocol_ids=pd.DataFrame(df_info['protocol_id'].unique())
protocol_ids.columns=['protocol_id']

df_total=df_project_weighted_stove_days.merge(df_info,left_on='protocol_id', right_on='protocol_id',how='left')
df_total.columns=['protocol_id','project_over_crediting','PoA','category','country','total_verified_credits','stove_days','lower','mean','upper','stove_days_mean_overcrediting']

df_total['over_credited_total']=df_total['total_verified_credits']/df_total['project_over_crediting']
total_verified_credits=df_total['total_verified_credits'].unique().sum()
total_over_crediting =df_total['over_credited_total'].unique().sum()
reported_value=total_verified_credits/total_over_crediting
print(total_verified_credits/total_over_crediting)
sd=df_total['total_verified_credits'].unique().std()/df_total['over_credited_total'].unique().std()
ci=sd*1.96/np.sqrt(51)
ci
ci_lower=reported_value-ci
ci_upper=reported_value+ci
print(reported_value,'[',ci_lower,',',ci_upper,']')

0    19.673103
dtype: object
13.975711510171877
13.975711510171877 [ 7.937443394357665 , 20.01397962598609 ]


In [15]:
df.head(3)

,protocol_type,protocol_version,PoA,protocol_id,monitoring_period,stove_year_cohort,total_verified_credits,type,verified_credits,verified_credits_per_stove,...,project_adoption_commercial,project_usage_commercial,leakage_binary,leakage_default,leakage_tCO2e,more_than_one_stove_adj,rebound,other_tCO2e_reduction,project_stacking,project_stacking_commercial
0,Units,0,0,0,0,0,tCO2e,0,tCO2e,tCO2e/stove/day,...,%,%,0,%,tCO2e,%,0.00,0,0.0,0
1,CDM AMS II G,AMS-II.G. ver. 9,10471,PoA 10471 CPA1,MR1,Domestic (2020),65200,firewood,65200,0.003652,...,0,0,1,0.95,0,1,0.78,0,0.0,0
2,CDM AMS II G,AMS-II.G. ver. 9,10471,PoA 10471 CPA2,MR1,Domestic (2020),63603,firewood,63603,0.003642,...,0,0,1,0.95,0,1,0.78,0,0.0,0


In [16]:
PoA_total_credits = df_total.groupby('PoA')['total_verified_credits'].sum()

In [17]:
# Assuming df_total DataFrame exists as defined in the provided code.
countries_with_PoA_8480 = df_total[df_total['PoA'] == 8480]['country'].unique()
countries_with_PoA_8480

array(['Guatemala'], dtype=object)

In [18]:
PoA_total_credits.sort_values(ascending=False)

,total_verified_credits
PoA,
10431,3074058
10471,2041596
9007,1325016
10415,746736
10443,421788
10477,408109
7359,335266.13
9956,295982
9265,283659


# By project

In [19]:
#df_project_weighted_stove_days_numerator['stove_days_mean_overcrediting']
#df_project_weighted_stove_days_denominator['stove_days']
#df_project_weighted_stove_days_numerator

In [20]:
df_project_weighted_stove_days_numerator=df_info.groupby('protocol_id')['stove_days_mean_overcrediting'].sum()
df_project_weighted_stove_days_numerator=pd.DataFrame(df_project_weighted_stove_days_numerator)


df_project_weighted_stove_days_denominator=df_info.groupby('protocol_id')['stove_days'].sum()
df_project_weighted_stove_days_denominator=pd.DataFrame(df_project_weighted_stove_days_denominator)
df_project_weighted_stove_days=df_project_weighted_stove_days_numerator['stove_days_mean_overcrediting']/df_project_weighted_stove_days_denominator['stove_days']
df_project_weighted_stove_days=pd.DataFrame(df_project_weighted_stove_days)
df_project_weighted_stove_days.mean() #This is the average project over-crediting
print(df_project_weighted_stove_days.mean())
df_project_weighted_stove_days
df_project_weighted_stove_days.to_excel("df_project_overcrediting.xlsx")

0    19.673103
dtype: object


In [21]:
df_project_weighted_stove_days

,0
protocol_id,
PoA 10008-CPA2,51.679697
PoA 10030-CPA 4,47.309176
PoA 10030-CPA3,24.505569
PoA 10030-CPA4,51.041224
PoA 10415-CPA1,29.318138
...,...
PoA 9956 CPA41,13.484039
PoA 9956 CPA42,12.776751
PoA 9956 CPA43,12.066305


# Total over-crediting

In [22]:
protocol_ids=pd.DataFrame(df_info['protocol_id'].unique())
protocol_ids.columns=['protocol_id']

df_total=df_project_weighted_stove_days.merge(df_info,left_on='protocol_id', right_on='protocol_id',how='left')
df_total.columns=['protocol_id', 'project_over_crediting','PoA','category','country','total_verified_credits','stove_days','lower','mean','upper','stove_days_mean_overcrediting']
df_total['over_credited_total']=df_total['total_verified_credits']/df_total['project_over_crediting']
total_verified_credits=df_total['total_verified_credits'].unique().sum()
total_over_crediting =df_total['over_credited_total'].unique().sum()
reported_value=total_verified_credits/total_over_crediting
print(total_verified_credits/total_over_crediting)
sd=df_total['total_verified_credits'].unique().std()/df_total['over_credited_total'].unique().std()
ci=sd*1.96/np.sqrt(len(protocol_ids))
ci
ci_lower=reported_value-ci
ci_upper=reported_value+ci
print(reported_value,'[',ci_lower,',',ci_upper,']')

13.975711510171877
13.975711510171877 [ 10.887691005921011 , 17.06373201442274 ]


In [ ]:
print(total_verified_credits)

7887173.973138815


In [ ]:
print(total_over_crediting)

564348.653547859


# PoA, country and Methodology-stove combination analysis

In [23]:
import pandas as pd
import numpy as np
from scipy import stats

def weighted_std(values, weights):
    """Calculate weighted standard deviation."""
    average = np.average(values, weights=weights)
    variance = np.average((values - average) ** 2, weights=weights)
    return np.sqrt(variance)

def weighted_std(values, weights):
    """Calculate weighted standard deviation."""
    average = np.average(values, weights=weights)
    variance = np.average((values - average) ** 2, weights=weights)
    return np.sqrt(variance)

def calculate_weighted_statistics(df, group_by_col):
    """
    Calculate weighted statistics grouped by specified column.

    Parameters:
    df (pandas.DataFrame): DataFrame with columns for grouping,
                         'total_verified_credits', and 'mean'
    group_by_col (str): Name of the column to group by

    Returns:
    pandas.DataFrame: Weighted statistics by group including:
                     - weighted average
                     - weighted standard deviation
                     - 95% confidence intervals
    """
    # Initialize empty lists to store results
    groups = []
    w_means = []
    w_stds = []
    lower_cis = []
    upper_cis = []

    # Calculate statistics for each group
    for group in df[group_by_col].unique():
        group_data = df[df[group_by_col] == group]
        weights = group_data['total_verified_credits']
        values = group_data['mean']

        # Calculate weighted mean
        w_mean = np.average(values, weights=weights)

        # Calculate weighted standard deviation
        w_std = weighted_std(values, weights)

        # Calculate standard error
        n = len(group_data)
        se = w_std / np.sqrt(n)

        # Calculate 95% confidence intervals
        ci = stats.t.interval(confidence=0.95, df=n-1, loc=w_mean, scale=se)

        # Store results
        groups.append(group)
        w_means.append(w_mean)
        w_stds.append(w_std)
        lower_cis.append(ci[0])
        upper_cis.append(ci[1])

    # Create results DataFrame
    results = pd.DataFrame({
        group_by_col: groups,
        'total': w_means,
        'sd': w_stds,
        'lower': lower_cis,
        'upper': upper_cis
    })

    # Round numeric columns to 4 decimal places
    numeric_columns = ['total', 'sd', 'lower', 'upper']
    results[numeric_columns] = results[numeric_columns].round(4)

    # Sort by group column
    results = results.sort_values(group_by_col).reset_index(drop=True)

    return results



In [24]:
df_PoA_final =calculate_weighted_statistics(df_total, 'PoA')
df_PoA_final

,PoA,total,sd,lower,upper
0,5341,14.5877,3.2060,12.9933,16.1820
1,5342,16.0655,7.2383,13.6164,18.5146
2,6864,16.5040,5.4466,10.7881,22.2198
3,7359,9.6857,0.6706,9.2596,10.1118
4,7997,26.0903,11.5895,20.1315,32.0491
5,8480,68.2925,4.5119,61.1131,75.4719
6,9007,19.8410,7.4664,16.9459,22.7362
7,9265,26.4457,5.7866,23.8116,29.0797
8,9956,21.5926,4.9896,20.3660,22.8192
9,10008,51.7149,1.2036,40.9012,62.5287


In [25]:
df_PoA_final.to_excel('all_factor_PoA_final.xlsx') ## SY added

In [26]:
df_country_final =calculate_weighted_statistics(df_total, 'country')
df_country_final

,country,total,sd,lower,upper
0,Bangladesh,20.5893,0.9380,19.8051,21.3735
1,Burundi,22.0672,1.5841,18.1320,26.0024
2,Cambodia,47.4071,2.4120,45.1764,49.6378
3,Fiji,54.2884,2.5622,50.2114,58.3653
4,Ghana,15.8914,3.7384,13.9694,17.8135
5,Guatemala,68.2925,4.5119,61.1131,75.4719
6,Kenya,15.2535,8.6567,13.0364,17.4706
7,Lao,24.5272,0.9830,23.4956,25.5588
8,Madagascar,47.3330,0.9161,46.9909,47.6751
9,Malawi,16.0147,3.8878,14.1408,17.8885


In [27]:
df_country_final.to_excel('all_factor_country_final.xlsx') ## SY added

In [28]:
df_combination_final=calculate_weighted_statistics(df_total, 'category')
df_combination_final

,category,total,sd,lower,upper
0,cdm_imp_charcoal,17.7052,8.5407,14.8576,20.5528
1,cdm_imp_ethanol,6.1275,1.7124,4.0012,8.2538
2,cdm_imp_firewood,23.2886,11.4513,22.1456,24.4316
3,ethanol,9.6335,0.9168,9.1042,10.1629


In [29]:
df_combination_final.to_excel('all_results_all_categories.xlsx')   ## SY updated

In [30]:
df_ALL_VERs_agw=pd.DataFrame(all_VERS_agw).transpose()
print(df_ALL_VERs_agw.shape[1])
ALL_i=0
df_ALL_VERs_agw_final=pd.DataFrame(np.zeros((1, 10000)))
while ALL_i<df_ALL_VERs_agw.shape[1]:
    df_ALL_VERs_agw_final=df_ALL_VERs_agw_final+df_ALL_VERs_agw[ALL_i]
    ALL_i=ALL_i+1
    print(ALL_i)




ALL_VERs=VERs_all

ALL_ratio_analysis_project=df_ALL_VERs_agw_final/sum(ALL_VERs.iloc[:,1])
ALL_ratio_analysis_project_mean=np.mean(df_ALL_VERs_agw_final, axis=1)/sum(ALL_VERs.iloc[:,1])
ALL_ratio_project_analysis_mean=ALL_ratio_analysis_project_mean**-1

ALL_sd_ratio_analysis_project=np.std(ALL_ratio_analysis_project, axis=1)



ALL_ratio_analysis_project_lower=ALL_ratio_analysis_project_mean - z*ALL_sd_ratio_analysis_project/np.sqrt(ALL_N)
ALL_ratio_analysis_project_upper=ALL_ratio_analysis_project_mean + z*ALL_sd_ratio_analysis_project/np.sqrt(ALL_N)

ALL_ratio_project_analysis_lower=ALL_ratio_analysis_project_lower**-1
ALL_ratio_project_analysis_upper=ALL_ratio_analysis_project_upper**-1

print(ALL_ratio_analysis_project_lower)
print(ALL_ratio_analysis_project_mean)
print(ALL_ratio_analysis_project_upper)
print(ALL_ratio_project_analysis_lower)
print(ALL_ratio_project_analysis_mean)
print(ALL_ratio_project_analysis_upper)

print(round(ALL_ratio_analysis_project_mean[0],10),'[',round(ALL_ratio_analysis_project_lower[0],10),',',round(ALL_ratio_analysis_project_upper[0],10),']')
print(round(ALL_ratio_project_analysis_mean[0],10),'[',round(ALL_ratio_project_analysis_upper[0],10),',',round(ALL_ratio_project_analysis_lower[0],10),']')

444
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276


In [31]:
df_all_VERS_agw=pd.DataFrame(df_ALL_VERs_agw_final)
df_all_VERS_agw.to_excel('df_all_VERs_agw_scenario_all.xlsx')

In [32]:
# df_gs_lpg_VERs_agw_final=np.array(df_gs_lpg_VERs_agw_final)
# df_gs_lpg_VERs_agw_final=df_gs_lpg_VERs_agw_final.flatten()

# df_GS_Meter_VERs_agw_final=np.array(df_GS_Meter_VERs_agw_final)
# df_GS_Meter_VERs_agw_final=df_GS_Meter_VERs_agw_final.flatten()

df_cdm_imp_firewood_VERs_agw_final=np.array(df_cdm_imp_firewood_VERs_agw_final)
df_cdm_imp_firewood_VERs_agw_final=df_cdm_imp_firewood_VERs_agw_final.flatten()

df_CDM_I_E_VERs_agw_final=np.array(df_CDM_I_E_VERs_agw_final)
df_CDM_I_E_VERs_agw_final=df_CDM_I_E_VERs_agw_final.flatten()

df_cdm_imp_charcoal_VERs_agw_final=np.array(df_cdm_imp_charcoal_VERs_agw_final)
df_cdm_imp_charcoal_VERs_agw_final=df_cdm_imp_charcoal_VERs_agw_final.flatten()

# df_gs_simple_firewood_VERs_agw_final=np.array(df_gs_simple_firewood_VERs_agw_final)
# df_gs_simple_firewood_VERs_agw_final=df_gs_simple_firewood_VERs_agw_final.flatten()


# df_gs_imp_firewood_VERs_agw_final=np.array(df_gs_imp_firewood_VERs_agw_final)
# df_gs_imp_firewood_VERs_agw_final=df_gs_imp_firewood_VERs_agw_final.flatten()

# df_gs_imp_charcoal_VERs_agw_final=np.array(df_gs_imp_charcoal_VERs_agw_final)
# df_gs_imp_charcoal_VERs_agw_final=df_gs_imp_charcoal_VERs_agw_final.flatten()

df_ALL_VERs_agw_final=np.array(df_ALL_VERs_agw_final)
df_ALL_VERs_agw_final=df_ALL_VERs_agw_final.flatten()

NameError: name 'df_cdm_imp_firewood_VERs_agw_final' is not defined

# Figure S1

In [33]:
category1 = ['All', 'GS-Metered-Pellets', 'GS-LPG', 'GS-Firewood','GS-Simplified Firewood','GS-Charcoal', 'CDM-Charcoal', 'CDM_Ethanol', 'CDM-Firewood', 'Ethanol'] # SY added  ethanol
category2 = list(df.country.unique()[1:])
category2.sort() # 알파벳 정렬
category3 = list(df_PoA_final.PoA.unique()[0:])
results_category = category1 + category2 + category3

In [34]:
results_lower = np.array([
    round(ci_lower, 2), 0, 0, 0, 0, 0,
                 round(df_combination_final['lower'][0],2),
                 round(df_combination_final['lower'][1],2),
                 round(df_combination_final['lower'][2],2),
                 round(df_combination_final['lower'][3],2),
                 round(df_country_final['lower'][0],2),
                 round(df_country_final['lower'][1],2),
                 round(df_country_final['lower'][2],2),
                 round(df_country_final['lower'][3],2),
                 round(df_country_final['lower'][4],2),
                 round(df_country_final['lower'][5],2),
                 round(df_country_final['lower'][6],2),
                 round(df_country_final['lower'][7],2),
                 round(df_country_final['lower'][8],2),
                 round(df_country_final['lower'][9],2),
                 round(df_country_final['lower'][10],2),
                 round(df_country_final['lower'][11],2),
                 round(df_country_final['lower'][12],2),
                 round(df_country_final['lower'][13],2),
                 round(df_country_final['lower'][14],2),
                 round(df_PoA_final['lower'][0],2),
                 round(df_PoA_final['lower'][1],2),
                 round(df_PoA_final['lower'][2],2),
                 round(df_PoA_final['lower'][3],2),
                 round(df_PoA_final['lower'][4],2),
                 round(df_PoA_final['lower'][5],2),
                 round(df_PoA_final['lower'][6],2),
                 round(df_PoA_final['lower'][7],2),
                 round(df_PoA_final['lower'][8],2),
                 round(df_PoA_final['lower'][9],2),
                 round(df_PoA_final['lower'][10],2),
                 round(df_PoA_final['lower'][11],2),
                 round(df_PoA_final['lower'][12],2),
                 round(df_PoA_final['lower'][13],2),
                 round(df_PoA_final['lower'][14],2),
                 round(df_PoA_final['lower'][15],2),
                 round(df_PoA_final['lower'][16],2),
                 round(df_PoA_final['lower'][17],2),
                 round(df_PoA_final['lower'][18],2),
                 round(df_PoA_final['lower'][19],2),
                 round(df_PoA_final['lower'][20],2)])

results_mean = np.array([
    round(reported_value, 2), 0, 0, 0, 0, 0,
                 round(df_combination_final['total'][0],2),
                 round(df_combination_final['total'][1],2),
                 round(df_combination_final['total'][2],2),
                 round(df_combination_final['total'][3],2),
                 round(df_country_final['total'][0],2),
                 round(df_country_final['total'][1],2),
                 round(df_country_final['total'][2],2),
                 round(df_country_final['total'][3],2),
                 round(df_country_final['total'][4],2),
                 round(df_country_final['total'][5],2),
                 round(df_country_final['total'][6],2),
                 round(df_country_final['total'][7],2),
                 round(df_country_final['total'][8],2),
                 round(df_country_final['total'][9],2),
                 round(df_country_final['total'][10],2),
                 round(df_country_final['total'][11],2),
                 round(df_country_final['total'][12],2),
                 round(df_country_final['total'][13],2),
                 round(df_country_final['total'][14],2),
                 round(df_PoA_final['total'][0],2),
                 round(df_PoA_final['total'][1],2),
                 round(df_PoA_final['total'][2],2),
                 round(df_PoA_final['total'][3],2),
                 round(df_PoA_final['total'][4],2),
                 round(df_PoA_final['total'][5],2),
                 round(df_PoA_final['total'][6],2),
                 round(df_PoA_final['total'][7],2),
                 round(df_PoA_final['total'][8],2),
                 round(df_PoA_final['total'][9],2),
                 round(df_PoA_final['total'][10],2),
                 round(df_PoA_final['total'][11],2),
                 round(df_PoA_final['total'][12],2),
                 round(df_PoA_final['total'][13],2),
                 round(df_PoA_final['total'][14],2),
                 round(df_PoA_final['total'][15],2),
                 round(df_PoA_final['total'][16],2),
                 round(df_PoA_final['total'][17],2),
                 round(df_PoA_final['total'][18],2),
                 round(df_PoA_final['total'][19],2),
                 round(df_PoA_final['total'][20],2)

])

results_high = np.array([
    round(ci_upper, 2), 0, 0, 0, 0, 0,
                 round(df_combination_final['upper'][0],2),
                 round(df_combination_final['upper'][1],2),
                 round(df_combination_final['upper'][2],2),
                 round(df_combination_final['upper'][3],2),
                 round(df_country_final['upper'][0],2),
                 round(df_country_final['upper'][1],2),
                 round(df_country_final['upper'][2],2),
                 round(df_country_final['upper'][3],2),
                 round(df_country_final['upper'][4],2),
                 round(df_country_final['upper'][5],2),
                 round(df_country_final['upper'][6],2),
                 round(df_country_final['upper'][7],2),
                 round(df_country_final['upper'][8],2),
                 round(df_country_final['upper'][9],2),
                 round(df_country_final['upper'][10],2),
                 round(df_country_final['upper'][11],2),
                 round(df_country_final['upper'][12],2),
                 round(df_country_final['upper'][13],2),
                 round(df_country_final['upper'][14],2),
                 round(df_PoA_final['upper'][0],2),
                 round(df_PoA_final['upper'][1],2),
                 round(df_PoA_final['upper'][2],2),
                 round(df_PoA_final['upper'][3],2),
                 round(df_PoA_final['upper'][4],2),
                 round(df_PoA_final['upper'][5],2),
                 round(df_PoA_final['upper'][6],2),
                 round(df_PoA_final['upper'][7],2),
                 round(df_PoA_final['upper'][8],2),
                 round(df_PoA_final['upper'][9],2),
                 round(df_PoA_final['upper'][10],2),
                 round(df_PoA_final['upper'][11],2),
                 round(df_PoA_final['upper'][12],2),
                 round(df_PoA_final['upper'][13],2),
                 round(df_PoA_final['upper'][14],2),
                 round(df_PoA_final['upper'][15],2),
                 round(df_PoA_final['upper'][16],2),
                 round(df_PoA_final['upper'][17],2),
                 round(df_PoA_final['upper'][18],2),
                 round(df_PoA_final['upper'][19],2),
                 round(df_PoA_final['upper'][20],2)])

In [35]:
# Create DataFrame

df_All_results_table = [results_lower,results_mean, results_high]
results_category = np.array(results_category)
x = pd.DataFrame(df_All_results_table)
x = x.transpose()
x['Category'] = results_category[:]
x
x.to_excel('df_all_factors_results_table.xlsx')

In [36]:
x

,0,1,2,Category
0,10.89,13.98,17.06,All
1,0.00,0.00,0.00,GS-Metered-Pellets
2,0.00,0.00,0.00,GS-LPG
3,0.00,0.00,0.00,GS-Firewood
4,0.00,0.00,0.00,GS-Simplified Firewood
5,0.00,0.00,0.00,GS-Charcoal
6,14.86,17.71,20.55,CDM-Charcoal
7,4.00,6.13,8.25,CDM_Ethanol
8,22.15,23.29,24.43,CDM-Firewood
9,9.10,9.63,10.16,Ethanol
